# Variational Autoencoder Galaxy generator

## Import libraries

In [ ]:
import os
from glob import glob
from utils.custom_utils import RetrieveData, show_images
import random as rd

from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

import wandb

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

## Download data

In [ ]:
DataURL='https://tinyurl.com/mr2yc5nx'

RetrieveData(DataURL)

## dataloader and data verification

In [ ]:
image_path='./data/images_gz2/images'

In [ ]:
class GalaxyZoo2(Dataset):
    def __init__(self, img_dir, transform=None, train=None):
        self.transform = transform
        self.img_dir = img_dir
        
        images = glob(os.path.join(img_dir,'*.jpg'))
        rd.shuffle(images)
        
        cut = int(0.6*len(images))
        if train==True:
            self.images = images[:cut]
        elif train==False:
            self.images = images[cut:]
        else:
            self.images = images
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img_path = self.images[index]
        with Image.open(img_path) as im:
            if self.transform:
                im = self.transform(im)
            return im.float()/255

In [ ]:
trainset=GalaxyZoo2(image_path, transform=transforms.PILToTensor())

In [ ]:
trainloader=DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

In [ ]:
images=next(iter(trainloader))
print(images.shape)
show_images(utils.make_grid(images))

## VAE

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'this instance use {device}')

torch.manual_seed(100)


image_size = 424
channels = 3
batch_size = 32

#hyperparameters of the NN specifically
input_features = channels*image_size*image_size # RGB pixel (dim=3)* 424*424 image
output_features = 128 # size of output of encoder = input of decoder
hidden_features = 200 # arbitrary
latent_space_size = 20 #size of the latent space --> arbitrary
learning_rate = 3e-4 # arbitrary
num_epochs = 10 # arbitrary

model_name = "MLP_VAE"
#directory = ""
#checkpoint = f"{directory}/{model_name}.pt"

In [ ]:
class MLP_VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, output_dim=128, latent_dim=2):
        super().__init__()
        
        self.encoder = nn.Sequential(nn.Linear(input_dim, hidden_dim),
                                     nn.ReLU(),
                                     nn.Linear(hidden_dim, output_dim),
                                    )
        
        self.mu = nn.Linear(output_dim, latent_dim)
        self.logvar = nn.Linear(output_dim, latent_dim)
        
        self.latent_mapping = nn.Linear(latent_dim, output_dim)
        
        self.decoder = nn.Sequential(nn.Linear(output_dim, hidden_dim),
                                     nn.ReLU(),
                                     nn.Linear(hidden_dim, input_dim),
                                    )
    
    def encode(self, x):
        encoder = self.encoder(x)
        mu, logvar = self.mu(encoder), self.logvar(encoder)
        return mu, logvar
    
    def sample_z(self, mu, logvar):
        eps = torch.rand_like(mu)
        return mu + eps * torch.exp(0.5 * logvar)
    
    def decode(self, z):
        latent_z = self.latent_mapping(z)
        out = self.decoder(latent_z)
        return torch.sigmoid(out)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.sample_z(mu, logvar)
        output = self.decode(z)
        return output, mu, logvar

In [ ]:
def ElboLoss(x, x_reconstructed, mu, logvar):
    criterion = F.binary_cross_entropy(x_reconstructed, x)
    KL_div = 0.5*torch.sum(1 + logvar - mu**2 - torch.exp(logvar))
    elbo = torch.mean(criterion - KL_div)
    return elbo

In [ ]:
trainset = GalaxyZoo2(image_path, transform=transforms.Compose([transforms.PILToTensor(),]), train=True)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
model = MLP_VAE(input_dim=input_features).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train(num_epochs):
    """
    Train the model (VAE) onto the data in the trainloader
    """
    train_avg_loss = []
    loop = tqdm(enumerate(trainloader))
    
    for i in range(num_epochs):
        train_losses = []
        loop
        
        for i,x in loop:
            x = x.to(device).view(x.shape[0],-1)
            x_reconstructed, mu, logvar = model(x)
        
            loss = ElboLoss(x, x_reconstructed, mu, logvar)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())
            train_losses.append(loss.detach())
                    
        train_avg_loss.append(torch.mean(torch.FloatTensor(train_losses)))

    return train_avg_loss #we could also return the reconstruction loss and the regularization loss individually, but in the end it is the ELBO loss that is important

In [ ]:
images=next(iter(trainloader))
test,mu,logvar=model(images.view(images.shape[0],-1))
show_images(utils.make_grid(test.view(test.shape[0],3,424,424)[0]))